In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from openpyxl import load_workbook

In [ ]:
# 1. Carregar dados
df_prices = pd.read_excel("stocks_train.xlsx", index_col=0, parse_dates=True)  # Lê os preços históricos das ações

In [ ]:
# 2. Calcular retornos diários
returns = df_prices.pct_change().dropna(how='any')  # Calcula os retornos percentuais diários e remove valores nulos

In [ ]:
# 3. Funções auxiliares
def annualized_return(weights, mean_daily_returns):
	# Calcula o retorno anualizado do portfólio
	return np.dot(weights, mean_daily_returns) * 252

def annualized_volatility(weights, cov_matrix):
	# Calcula a volatilidade anualizada do portfólio
	return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)

def negative_sharpe_ratio(weights, mean_daily_returns, cov_matrix, risk_free_rate=0.0):
	# Calcula o Sharpe Ratio negativo (para ser minimizado)
	ret = annualized_return(weights, mean_daily_returns)
	vol = annualized_volatility(weights, cov_matrix)
	return -(ret - risk_free_rate) / vol

In [ ]:
# 4. Preparar variáveis para otimização
mean_returns = returns.mean()  # Média dos retornos diários de cada ativo
cov_matrix = returns.cov()     # Matriz de covariância dos retornos
num_assets = len(mean_returns) # Número de ativos

constraints = ({"type": "eq", "fun": lambda x: np.sum(x) - 1},)  # Restrição: soma dos pesos = 1
bounds = tuple((0, 1) for _ in range(num_assets))                 # Limites: cada peso entre 0 e 1
initial_guess = num_assets * [1. / num_assets]                    # Chute inicial: pesos iguais

In [ ]:
# 5. Otimização
opt_result = minimize(
	negative_sharpe_ratio,           # Função objetivo (Sharpe negativo)
	initial_guess,                   # Chute inicial
	args=(mean_returns, cov_matrix), # Argumentos extras para a função objetivo
	method='SLSQP',                  # Algoritmo de otimização
	bounds=bounds,                   # Limites dos pesos
	constraints=constraints          # Restrições
)

weights_opt = opt_result.x           # Vetor de pesos otimizados
tickers = mean_returns.index         # Lista dos tickers dos ativos

In [ ]:
# 6. Exportar no formato solicitado
from openpyxl import Workbook

wb = Workbook()                      # Cria um novo arquivo Excel
ws = wb.active                       # Seleciona a planilha ativa

ws["A1"] = "Código"                  # Escreve o cabeçalho "Código" na célula A1
ws["A2"] = "Peso"                    # Escreve o cabeçalho "Peso" na célula A2

# Escreve os tickers e pesos nas colunas B até CW
for i, (ticker, peso) in enumerate(zip(tickers, weights_opt), start=2):  # coluna B = 2
	col_letter = ws.cell(row=1, column=i).column_letter  # Obtém a letra da coluna
	ws[f"{col_letter}1"] = ticker                       # Escreve o ticker na linha 1
	ws[f"{col_letter}2"] = float(peso)                  # Escreve o peso na linha 2

wb.save("PesosOtimizados.xlsx")      # Salva o arquivo Excel com os pesos otimizados
